In [1]:
from models.Model import Model
from models.Logistic.MultiGPT2Logistic import MultiGPT2Logistic
from languages.LanguageModel import LanguageModel
from DataExploration import DataExploration
# from languages.Japanese import Japanese
from languages.English import English
from languages.Finnish import Finnish
from Preprocess import Preprocess
from Pipeline import Pipeline
from typing import List
import torch
import datasets
import pandas as pd
import random
import numpy as np

In [2]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently 
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)

enforce_reproducibility()

In [3]:
# Is used to minimize the clutter in the console
datasets.logging.set_verbosity_error()

# Define the languages to be used
languages: List[LanguageModel] = [
    English(),
    Finnish(),
    # Japanese()
]

# gpt2Generator = GPT2Generator()
torch.cuda.empty_cache()


# Preprocessing and Dataset Analysis

In [4]:
all_data = {}

for language in languages:
    pipeline = Pipeline()

    # Get the preprocessed data and split it into training and validation data
    preprocessor = Preprocess(language.tokenize, language.clean)
    data = pipeline.get_data(language=language.name, preproccesor=preprocessor)
    train_data, validation_data = pipeline.split_data(data)

    all_data[language.name] = {
        "train_data": train_data,
        "validation_data": validation_data
    }


Loading data...

Loading data...


# Multilingual QA

We start by implementing our binary question system with a multilingual encoder instead of the monolingual ones. With this we perform zero-shot cross-lingual evaluation

In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [6]:
# Zero shot classification
for train_language in languages:
    print(f'\n\n--- Training on {train_language.name} ---')
    model = MultiGPT2Logistic()
    model.set_language(train_language.name)
    
    try:
        model.load()
    except:
        train_data = all_data[train_language.name]["train_data"]
        train_data = train_data.head(10)
        X_train = model.extract_X(train_data)
        y_train = train_data['is_answerable']
        model = pipeline.train(
            model,
            X_train,
            y_train
        )
        model.save()
    
    for val_language in languages:
        print(f'\n\t- Validating on {val_language.name}')
        validation_data = all_data[val_language.name]["validation_data"]
        validation_data = validation_data.head(10)
        X_validation = model.extract_X(validation_data)
        y_validation = validation_data['is_answerable']
        pipeline.evaluate(
            model,
            X_validation,
            y_validation
        )



--- Training on english ---

	- Validating on english


Using eos_token, but it is not set yet.
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using eos_token, but it is not set yet

  0%|          | 0/10 [00:00<?, ?ex/s]

Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained("bert-base-multilingual-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

loading file vocab.txt from cache at C:\Users\chris/.cache\huggingface\hub\models--bert-base-multilingual-uncased\snapshots\800c34f3d5aa174fe531f560b44b8d14592225b7\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\chris/.cache\huggingface\hub\models--bert-base-multilingual-uncased\snapshots\800c34f3d5aa174fe531f560b44b8d14592225b7\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\chris/.cache\huggingface\hub\models--bert-base-multilingual-uncased\snapshots\800c34f3d5aa174fe531f560b44b8d14592225b7\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02

In [ ]:
output["last_hidden_state"].shape

torch.Size([1, 12, 768])